In [21]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    balanced_accuracy_score,
    f1_score
)

import joblib

## 1) Paths + output folders
Kept everything relative to the project root so it runs anywhere (local, Colab, etc.)
- data/      : raw CSV files
- models/    : saved ML models
- outputs/   : rankings + any generated files for the Streamlit app

In [22]:
PROJECT_ROOT = Path(".").resolve()
DATA_DIR = PROJECT_ROOT / "data"
MODELS_DIR = PROJECT_ROOT / "models"
OUTPUTS_DIR = PROJECT_ROOT / "outputs"

MODELS_DIR.mkdir(exist_ok=True)
OUTPUTS_DIR.mkdir(exist_ok=True)


## 2) Load review files (chunked across multiple CSVs)

The Kaggle dataset often splits reviews into multiple files like:
reviews_0-25000.csv, reviews_25000-50000.csv, etc.
This collects them and concatenates into one DataFrame


In [23]:
review_files = sorted(DATA_DIR.glob("reviews_*.csv"))
review_files


[PosixPath('/Users/sanskritimalakar/Sephora-Product-Rating-Predictor/data/reviews_0-250.csv'),
 PosixPath('/Users/sanskritimalakar/Sephora-Product-Rating-Predictor/data/reviews_1250-end.csv'),
 PosixPath('/Users/sanskritimalakar/Sephora-Product-Rating-Predictor/data/reviews_250-500.csv'),
 PosixPath('/Users/sanskritimalakar/Sephora-Product-Rating-Predictor/data/reviews_500-750.csv'),
 PosixPath('/Users/sanskritimalakar/Sephora-Product-Rating-Predictor/data/reviews_750-1250.csv')]

In [24]:
dfs = []
for fp in review_files:
    df_part = pd.read_csv(fp, low_memory=False)
    df_part.columns = [c.strip() for c in df_part.columns]
    dfs.append(df_part)

reviews = pd.concat(dfs, ignore_index=True)
print("Reviews shape:", reviews.shape)
reviews.head(5)


Reviews shape: (1094411, 19)


,Unnamed: 0,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,0,1741593524,5,1.0,1.0,2,0,2,2023-02-01,I use this with the Nudestix “Citrus Clean Bal...,Taught me how to double cleanse!,NaN,brown,dry,black,P504322,Gentle Hydra-Gel Face Cleanser,NUDESTIX,19.0
1,1,31423088263,1,0.0,NaN,0,0,0,2023-03-21,I bought this lip mask after reading the revie...,Disappointed,NaN,NaN,NaN,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
2,2,5061282401,5,1.0,NaN,0,0,0,2023-03-21,My review title says it all! I get so excited ...,New Favorite Routine,light,brown,dry,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
3,3,6083038851,5,1.0,NaN,0,0,0,2023-03-20,I’ve always loved this formula for a long time...,Can't go wrong with any of them,NaN,brown,combination,black,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
4,4,47056667835,5,1.0,NaN,0,0,0,2023-03-20,"If you have dry cracked lips, this is a must h...",A must have !!!,light,hazel,combination,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0


## 3) Load product metadata
This gives us product name, brand, and usually price.
We later merge reviews -> products using product_id.

In [25]:
products = pd.read_csv(DATA_DIR / "product_info.csv", low_memory=False)
products.columns = [c.strip() for c in products.columns]
print("Products shape:", products.shape)
products.head(5)


Products shape: (8494, 27)


,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,size,variation_type,variation_value,...,online_only,out_of_stock,sephora_exclusive,highlights,primary_category,secondary_category,tertiary_category,child_count,child_max_price,child_min_price
0,P473671,Fragrance Discovery Set,6342,19-69,6320,3.6364,11.0,NaN,NaN,NaN,...,1,0,0,"['Unisex/ Genderless Scent', 'Warm &Spicy Scen...",Fragrance,Value & Gift Sets,Perfume Gift Sets,0,NaN,NaN
1,P473668,La Habana Eau de Parfum,6342,19-69,3827,4.1538,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,85.0,30.0
2,P473662,Rainbow Bar Eau de Parfum,6342,19-69,3253,4.2500,16.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
3,P473660,Kasbah Eau de Parfum,6342,19-69,3018,4.4762,21.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
4,P473658,Purple Haze Eau de Parfum,6342,19-69,2691,3.2308,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0


## 4) Merge reviews + products
Left join keeps every review even if product metadata is missing for some items.
`suffixes` avoids column name collisions after merge.

In [26]:
df = reviews.merge(
    products,
    on="product_id",
    how="left",
    suffixes=("_review", "_product")
)

print("Merged shape:", df.shape)
df.columns


Merged shape: (1094411, 45)


Index(['Unnamed: 0', 'author_id', 'rating_review', 'is_recommended',
       'helpfulness', 'total_feedback_count', 'total_neg_feedback_count',
       'total_pos_feedback_count', 'submission_time', 'review_text',
       'review_title', 'skin_tone', 'eye_color', 'skin_type', 'hair_color',
       'product_id', 'product_name_review', 'brand_name_review',
       'price_usd_review', 'product_name_product', 'brand_id',
       'brand_name_product', 'loves_count', 'rating_product', 'reviews',
       'size', 'variation_type', 'variation_value', 'variation_desc',
       'ingredients', 'price_usd_product', 'value_price_usd', 'sale_price_usd',
       'limited_edition', 'new', 'online_only', 'out_of_stock',
       'sephora_exclusive', 'highlights', 'primary_category',
       'secondary_category', 'tertiary_category', 'child_count',
       'child_max_price', 'child_min_price'],
      dtype='object')

## 5) Column cleanup / standardization
The merge can create columns like rating_review, brand_name_review, etc. Here we rename to clean "canonical" names used later.

In [27]:
df = df.rename(columns={
    "rating_review": "rating",
    "product_name_review": "product_name",
    "brand_name_review": "brand_name",
    "price_usd_review": "price_usd"
})

# fallback if merge created _x/_y instead
if "rating" not in df.columns and "rating_x" in df.columns:
    df["rating"] = df["rating_x"]
if "product_name" not in df.columns and "product_name_x" in df.columns:
    df["product_name"] = df["product_name_x"]
if "brand_name" not in df.columns and "brand_name_x" in df.columns:
    df["brand_name"] = df["brand_name_x"]
if "price_usd" not in df.columns and "price_usd_x" in df.columns:
    df["price_usd"] = df["price_usd_x"]

df = df.drop(columns=["Unnamed: 0"], errors="ignore")

df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
df = df.dropna(subset=["review_text", "rating", "product_name", "brand_name"]).copy()

df.shape


(1092967, 44)

## 7) Clean the review text
Goal: normalize text so TF-IDF is more useful.
Steps:
- lowercase
- remove HTML tags
- remove URLs
- keep only letters/numbers/spaces/apostrophes
- collapse extra whitespace

In [28]:
def clean_text(s: str) -> str:
    s = str(s).lower()
    s = re.sub(r"<.*?>", " ", s)
    s = re.sub(r"http\S+|www\.\S+", " ", s)
    s = re.sub(r"[^a-z0-9\s']", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["review_text_clean"] = df["review_text"].map(clean_text)
df[["rating", "review_text_clean"]].head(5)


,rating,review_text_clean
0,5,i use this with the nudestix citrus clean balm...
1,1,i bought this lip mask after reading the revie...
2,5,my review title says it all i get so excited t...
3,5,i ve always loved this formula for a long time...
4,5,if you have dry cracked lips this is a must ha...


## 8) Convert star rating -> sentiment label

We’re using star rating as a proxy target (weak supervision):
- 1–2: negative
- 3:   neutral
- 4–5: positive

In [29]:
def rating_to_sentiment(r):
    if r <= 2:
        return "negative"
    elif r == 3:
        return "neutral"
    else:
        return "positive"

df["sentiment"] = df["rating"].map(rating_to_sentiment)
df["sentiment"].value_counts(normalize=True)


sentiment
positive    0.820843
negative    0.104359
neutral     0.074798
Name: proportion, dtype: float64

## 9) Train/test split
- stratify keeps the class distribution consistent in train and test
- random_state ensures reproducibility

In [30]:
X_train, X_test, y_train, y_test = train_test_split(
    df["review_text_clean"],
    df["sentiment"],
    test_size=0.2,
    random_state=42,
    stratify=df["sentiment"]
)



## 10) Model: TF-IDF + Logistic Regression

Why this works well:
- TF-IDF (with bigrams) captures lots of sentiment phrasing ("love it", "broke me out")
- Logistic regression is fast and usually strong for text classification
- class_weight="balanced" helps with imbalanced labels

In [31]:
model = Pipeline([
    ("tfidf", TfidfVectorizer(
        max_features=150_000,
        ngram_range=(1, 2),
        min_df=3
    )),
    ("clf", LogisticRegression(
        max_iter=2000,
        class_weight="balanced"
    ))
])

model.fit(X_train, y_train)

,steps,"[('tfidf', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


11) Evaluate on the test set

Print:
- classification report (precision/recall/F1 per class)
- balanced accuracy (better than regular accuracy for imbalanced classes)
- confusion matrix to see what gets confused with what

In [32]:
preds = model.predict(X_test)

print(classification_report(y_test, preds))
print("Balanced accuracy:", balanced_accuracy_score(y_test, preds))
print("Confusion matrix (neg, neutral, pos):")
print(confusion_matrix(y_test, preds, labels=["negative","neutral","positive"]))


              precision    recall  f1-score   support

    negative       0.76      0.81      0.78     22812
     neutral       0.40      0.68      0.51     16351
    positive       0.99      0.92      0.95    179431

    accuracy                           0.89    218594
   macro avg       0.72      0.80      0.75    218594
weighted avg       0.92      0.89      0.90    218594

Balanced accuracy: 0.8015920348011513
Confusion matrix (neg, neutral, pos):
[[ 18373   4012    427]
 [  3374  11181   1796]
 [  2498  12654 164279]]


## 12) Predict sentiment for ALL reviews
Create:
- pred_sentiment: model output for every review
- is_positive: binary indicator (1 if predicted positive else 0)

In [33]:
df["pred_sentiment"] = model.predict(df["review_text_clean"])
df["is_positive"] = (df["pred_sentiment"] == "positive").astype(int)

## 13) Helper: safe weighted averages
Optionally weight each review by "helpfulness" if available.
If weights are missing / all zero, fall back to a simple mean.


In [34]:
def safe_weighted_average(values, weights):
    values = np.asarray(values, dtype=float)
    weights = np.asarray(weights, dtype=float)
    weights = np.nan_to_num(weights, nan=0.0)
    if weights.sum() <= 0:
        return float(np.nanmean(values))
    return float(np.average(values, weights=weights))

if "helpfulness" in df.columns:
    df["helpfulness"] = pd.to_numeric(df["helpfulness"], errors="coerce")
    df["helpfulness_weight"] = df["helpfulness"].clip(0, 1).fillna(0.0)
else:
    df["helpfulness_weight"] = 1.0

## 15) Bayesian smoothing + "worth it" scoring

Problem:
- products with very few reviews can look unrealistically good (or bad)

Fix:
- Bayesian adjustment pulls small-sample products toward the global mean
- As n_reviews grows, the adjusted score moves closer to the product’s true mean

In [35]:
def bayesian_adjusted(mean, n, global_mean, m=50):
    return (n / (n + m)) * mean + (m / (n + m)) * global_mean

global_rating = float(df["rating"].mean())
global_pos_rate = float(df["is_positive"].mean())

## 16) Aggregate to the product level

For each product, compute:
- n_reviews
- avg_rating (weighted)
- pos_rate  (weighted)
- price_usd (mean, if available)

In [36]:
grouped = (
    df.groupby(["product_id", "product_name", "brand_name"], as_index=False)
      .apply(lambda g: pd.Series({
          "n_reviews": len(g),
          "avg_rating": safe_weighted_average(g["rating"], g["helpfulness_weight"]),
          "pos_rate": safe_weighted_average(g["is_positive"], g["helpfulness_weight"]),
          "price_usd": float(np.nanmean(pd.to_numeric(g.get("price_usd"), errors="coerce"))) if "price_usd" in g else np.nan
      }))
      .reset_index(drop=True)
)

/var/folders/yy/dhj_4zq95fjc0lp4rlgtp80c0000gn/T/ipykernel_10441/3623727931.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


## 17) Filter to products with enough reviews + Apply Bayesian smoothing

This prevents rankings from being dominated by products with tiny sample sizes.


In [37]:
MIN_REVIEWS = 25
M = 50
grouped = grouped[grouped["n_reviews"] >= MIN_REVIEWS].copy()

grouped["adj_rating"] = grouped.apply(lambda r: bayesian_adjusted(r["avg_rating"], int(r["n_reviews"]), global_rating, m=M), axis=1)
grouped["adj_pos_rate"] = grouped.apply(lambda r: bayesian_adjusted(r["pos_rate"], int(r["n_reviews"]), global_pos_rate, m=M), axis=1)
grouped["adj_rating_norm"] = ((grouped["adj_rating"] - 1) / 4).clip(0, 1)


## 18) Value score from price + Worth it score

If price exists, we scale price into a 0–1 "value_score" where:
- cheapest products → value_score close to 1
- most expensive → value_score close to 0

Weighted blend:
- 40% adjusted (smoothed) rating
- 40% adjusted positive sentiment rate
- 20% value score (price-based)

In [38]:
if grouped["price_usd"].notna().any() and grouped["price_usd"].max() != grouped["price_usd"].min():
    grouped["value_score"] = 1 - (grouped["price_usd"] - grouped["price_usd"].min()) / (grouped["price_usd"].max() - grouped["price_usd"].min())
    grouped["value_score"] = grouped["value_score"].clip(0, 1)
else:
    grouped["value_score"] = 0.5

grouped["worth_it_score"] = (
    0.4 * grouped["adj_rating_norm"] +
    0.4 * grouped["adj_pos_rate"] +
    0.2 * grouped["value_score"]
)

rankings = grouped.sort_values("worth_it_score", ascending=False).reset_index(drop=True)
rankings.head(10)

,product_id,product_name,brand_name,n_reviews,avg_rating,pos_rate,price_usd,adj_rating,adj_pos_rate,adj_rating_norm,value_score,worth_it_score
0,P504429,Hydrating Serum,SEPHORA COLLECTION,300.0,5.000000,1.000000,20.00,4.899879,0.965947,0.974970,0.961883,0.968743
1,P505024,The POREfessional Get Unblocked Makeup-Removin...,Benefit Cosmetics,477.0,5.000000,1.000000,39.00,4.933506,0.977384,0.983376,0.919283,0.968161
2,P504575,PRO ACV Healthy Gut Vegan Gummies with 100% Or...,HUM Nutrition,168.0,5.000000,1.000000,26.00,4.839255,0.945327,0.959814,0.948430,0.951743
3,P173726,Facial Cotton,Shiseido,2820.0,4.785280,0.909017,13.00,4.776811,0.906449,0.944203,0.977578,0.935776
4,P503197,Juneberry & Collagen Hydrating Cold Cream Clea...,alpyn beauty,240.0,4.804548,0.969686,39.00,4.717411,0.933814,0.929353,0.919283,0.929123
5,P502986,Jade Facial Roller,SEPHORA COLLECTION,129.0,4.891473,0.961240,22.00,4.726020,0.905482,0.931505,0.957399,0.926275
6,P501188,SATOCANE Pore Purifying Scrub Mask,WASO,494.0,4.865672,0.910448,38.00,4.813602,0.896769,0.953400,0.921525,0.924373
7,P505233,Honey Infused Lip Oil,Gisou,81.0,5.000000,1.000000,32.00,4.732501,0.909018,0.933125,0.934978,0.923853
8,P505711,Polyglutamic Acid Dewy Sunscreen SPF 30,The INKEY List,100.0,4.750000,1.000000,14.99,4.599718,0.920542,0.899929,0.973117,0.922812
9,P505338,Pore Perfecting Liquid Exfoliator with 2% BHA ...,alpyn beauty,100.0,4.944645,0.986161,39.00,4.729481,0.911316,0.932370,0.919283,0.921331


## 19) Quick error analysis

Pull a random sample of misclassifications to see what the model struggles with.
This is useful for improving cleaning, labels, or adding more features.

In [39]:
errors = pd.DataFrame({"text": X_test, "true": y_test.values, "pred": preds})
errors[errors["true"] != errors["pred"]].sample(10, random_state=42)

,text,true,pred
1043157,this is probably one of the better cleansing b...,positive,neutral
546224,at first i loved using this toner but after a ...,positive,negative
782472,this primer works fabulous if you don t use a ...,neutral,positive
240725,i enjoyed the texture of this cream and found ...,positive,neutral
328533,i really wanted to love this but overall i did...,negative,neutral
470508,this was one of the first product i bought fro...,positive,negative
819197,so far i have found that this has a positive e...,neutral,positive
440922,this product is nice it definitely reduced app...,neutral,positive
188810,a very solid lip balm my lips are definitely b...,positive,neutral
1023424,i bought this as a moisturizer substitute beca...,positive,neutral


## 20) Save outputs for your Streamlit app
- sentiment_model.joblib: used for inference later (predicting sentiment)
- product_rankings.csv: used to display top products and filters in the UI

In [40]:
joblib.dump(model, MODELS_DIR / "sentiment_model.joblib")
rankings.to_csv(OUTPUTS_DIR / "product_rankings.csv", index=False)
print("Saved model + rankings")


Saved model + rankings
